In [ ]:
!pip install autotrain-advanced
!pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 104.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 71.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 304.5/304.5 kB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 113.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB

In [ ]:
!autotrain setup --update-torch

> INFO    Installing latest xformers
> INFO    Successfully installed latest xformers
> INFO    Installing latest PyTorch
> INFO    Successfully installed latest PyTorch


In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!autotrain llm --train \
    --project_name "ko-llama2-finetune" \
    --model "TinyPixel/Llama-2-7B-bf16-sharded" \
    --data_path "royboy0416/ko-alpaca" \
    --text_column "text" \
    --use_peft \
    --use_int4 \
    --learning_rate 3e-4 \
    --train_batch_size 8 \
    --num_train_epochs 2 \
    --trainer sft \
    --model_max_length 1024


> INFO    Running LLM
> INFO    Params: Namespace(version=False, train=True, deploy=False, inference=False, data_path='royboy0416/ko-alpaca', train_split='train', valid_split=None, text_column='text', rejected_text_column='rejected', prompt_text_column='prompt', model='TinyPixel/Llama-2-7B-bf16-sharded', model_ref=None, learning_rate=0.0003, num_train_epochs=2, train_batch_size=8, warmup_ratio=0.1, gradient_accumulation_steps=1, optimizer='adamw_torch', scheduler='linear', weight_decay=0.0, max_grad_norm=1.0, seed=42, add_eos_token=False, block_size=-1, use_peft=True, lora_r=16, lora_alpha=32, lora_dropout=0.05, logging_steps=-1, project_name='ko-llama2-finetune', evaluation_strategy='epoch', save_total_limit=1, save_strategy='epoch', auto_find_batch_size=False, fp16=False, push_to_hub=False, use_int8=False, model_max_length=1024, repo_id=None, use_int4=True, trainer='sft', target_modules=None, merge_adapter=False, token=None, backend='default', username=None, use_flash_attention_2=Fal

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: ignored

In [ ]:
!cp -a /content/ko-llama2-finetune /content/drive/MyDrive

In [ ]:
!pip install -q transformers peft bitsandbytes

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

model_id = "TinyPixel/Llama-2-7B-bf16-sharded"
peft_model_id = "/content/drive/MyDrive/ko-llama2-finetune/checkpoint-2460"

config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
    load_in_8bit=False,
    load_in_4bit=True,
    llm_int8_threshold=6.0,
    llm_int8_skip_modules=None,
    llm_int8_enable_fp32_cpu_offload=False,
    llm_int8_has_fp16_weight=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype="float16",
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

model.eval()

ModuleNotFoundError: ignored

In [3]:
prompt = "Below is an instruction that describes a task. Write a response that appropriately completes the request. ### Instruction: %s ### Response: "

def gen(x):
    q = prompt % (x,)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=128,
        early_stopping=True,
        do_sample=True,
    )
    return tokenizer.decode(gened[0]).replace(q, "")

In [ ]:
gen("태풍이 오면 어떻게 해야하나요?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


'<s> 태풍이라면 안전적인 처방 및 지원 인명목록을 제공해야합니다.</s>'

In [ ]:
gen("과제를 쉽게 하려면 어떻게 해야하나요?")

'<s> 솔루션을 개발하기란 쉽지 않고, 문제점을 관리함으로써 효율적으로 효과적으로 솔루션을 도출하는 것이 중요합니다. 이를 위해서는 해결책에 대'

In [ ]:
gen("숙제를 쉽게 하려면 어떻게 해야하나요?")

'<s> 숙제를 쉽게 하려면 이야기를 만들기 시작하고 숙제를 구성하고, 목표를 완료할 때까지 일일이 예시를 만들 것입니다.매일 질문에 대한 가치가 있으면 쉽'

In [ ]:
gen("라면끓이는 방법을 쉽게 설명해줘")

'<s> 라면을 끓잖아, 라면을 조금 쌓은 후, 물담자 속에 뚜균의 충전을 통해 라면을 끓잔다. 그리고, 바를 바운, 꿀병합니다.</s>'

In [ ]:
gen("구글 콜랩에서 이 파일을 손실되지 않게 하고싶은데 얻젛게 해야하는지 길게 설명해줘")

'<s> 큰 피치와 함께 설명하고, 청사진 및 그믅 요구사항을 목록으로 구성한 후, 모든 해결 장소를 살펴보는 것이 적합합니다. 이 방법은 대화형 구글 '

In [ ]:
gen("너는 왜이리 짧게 설명해줘?")

'<s> 너는 짧게 설명해주는 이유는 가독성이 있도록 정리된 단어와 구문이 잘 지적어질 수 있기 때문입니다.</s>'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
gen("너는 왜이리 길게  설명안해줘?")

'<s> 어떤 문장이 왜이 길게 만들어지는지에 대해서 말이 없습니다.</s>'

In [ ]:
gen("신기하지 않아?")

In [ ]:
gen("신기하지 않아?")

'<s> 새로운 기술의 작동 방식과 기능, 제품 또는 서비스의 장점 또는 이점을 설명하십시오.</s>'

In [ ]:
gen("신기하지 않아?")

'<s> 충분히 출력되어야 하므로 아래와 같은 입력이 주어졌을 때, 추가적인 필요한 정보를 제공하여(예를 들면 컴퓨터의 설정 폭과 크기 등 여타 정보'

In [ ]:
gen("회로도가 신기하지 않아?")

'<s> 회로도가 신기하게 보이는 이유는, 그것은 전기 신호를 둘러싸고 쌍곡선 콜라번슬을 만드는 데 중요한 역할을 하므로 이른바 "쌍곡선 테마"를 나타냅니다.</s>'

In [ ]:
gen("회로도에 대해서 알려줘")

'<s> 회로도(Circuit Diagram)는 전기 회로나 전자 회로의 구조를 그리기 위해 사용되는 전기 수학적 표현입니다. 각 회로와 기능 종류의 회로 요소를 기술한 도표를 알아보십시오.</s>'

In [ ]:
gen("앞으로 뭘하면 좋을지 말해줘")

'<s> 앞으로는 마음 그대로 할 수 있는 건 쇼 패션을 만들어보세요!</s>'

In [32]:
gen("노트북  뭐사면 좋을까?")

'<s> 올바른 문장 구성을 위해서 몇 가지 사이트 추가가 필요할 수도 있습니다. 노트북 전문가 포럼, 저렴한곳 찾기, 공통적인 상호 참여 웹사이트와 같은 일부'

In [34]:
gen("hello")

'<s> 안녕하세요.</s>'

In [37]:
gen("what is your name?")

'<s> 저는 이이하 알리야, 더욱 탁월한 지명</s>'

In [38]:
gen("뭐먹을까?")

'<s> 지난주에 관왕본, 아가씨, 글로버, 마법광충, 토란, 잡효료 등 다양한 식물을 소비하고 있음. 분량에 따라 요리를 판단시켜야합니다.</s>'

In [4]:
gen("뭐먹을까?")

NameError: ignored